In [1]:
import json, os
from google.cloud import storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "CREDENTIALS_PATH" # TODO: replace with your own credentials path
# Upload to GCS
bucket_name = "BUCKET_NAME" # TODO: replace with your own bucket name
local_dir = "LOCAL_DIR" # TODO: replace with your own local directory
prefix = "PREFIX" # TODO: replace with your own prefix

In [5]:
def process_to_gcp(file_path, name):
    # load 
    with open(file_path, 'r') as f:
        data = json.load(f)

    # get all seqs
    if "hsp" not in name:
        seqs = [d['search_path'] for d in data]
    else:
        def add_all_calls(trace_dict):
            all_calls = []
            all_calls += trace_dict['main_calls']
            for sub in trace_dict['sub_calls']:
                for sub_trace in sub:
                    all_calls += add_all_calls(sub_trace)
            return all_calls
        seqs = []
        for dp in data:
            seqs += add_all_calls(dp['trace_dict'])
    print(f"name: {len(seqs)}")

    # local save
    os.makedirs(local_dir, exist_ok=True)
    local_path = os.path.join(local_dir, f"{name}.json")
    with open(local_path, "w") as f:
        json.dump(seqs, f)

    # send to gcp
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    # # Upload train file
    train_blob = bucket.blob(prefix+f"{name}.json")
    train_blob.upload_from_filename(local_path)

In [6]:
todos = [
    ["./train_apr.json", "train_apr"],
    # TODO: add your own data here
]

In [ ]:
from tqdm import tqdm
for file_path, name in tqdm(todos):
    process_to_gcp(file_path, name)